# Goal : Scan the 3D predcitions with the Dpo fold database

### A. Run Foldseek
### B. Get the hits with good proba
### C. Report the folds
***

In [ ]:
rsync -avzhe ssh \
conchae@garnatxa.srv.cpd:/home/conchae/PhageDepo_pdb/Dpo_domains.phagedepo.0805.final.tsv \
/media/concha-eloko/Linux/depolymerase_building

***
## A  

In [ ]:
import subprocess
import os

path_project = "/media/concha-eloko/Linux/depolymerase_building"
path_pdb = f"{path_project}/phagedepo_out"
path_db = f"/media/concha-eloko/Linux/depolymerase_building/RefDepo_domains/RefDepo_domains_db"
path_tmp = f"{path_project}/tmp"

def seek_beta_helix(path_in) :
    dir_out = f"{path_project}/seekfold_phagedepo"
    protein_id = path_in.split("/")[-1].split(".pdb")[0]
    path_out = f"{dir_out}/{protein_id}.out"
    output_frmt = f"query,target,pident,alnlen,gapopen,qstart,qend,tstart,tend,bits,prob"
    seek = f"foldseek easy-search {path_in} {path_db} {path_out} {path_tmp} --format-output {output_frmt}"
    #seek = f"foldseek easy-search {path_in} {path_db} {path_out}.html {path_tmp} --format-mode 3"
    seek_process = subprocess.Popen(seek, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    press_out, press_err = seek_process.communicate()
    print (press_out, press_err)

paths = [f"{path_pdb}/{file}" for file in os.listdir(f"{path_pdb}")]

for path in paths :
    seek_beta_helix(path)

In [11]:
import pandas as pd 

phagedepo_df = pd.read_csv(f"{path_project}/Dpo_domains.phagedepo.0805.final.tsv", header = 0, sep = "\t")

phagedepo_df

,Seq_ID,Fold,Prob,Boundaries,Dpo_seq,Full_seq
0,phagedepo__3863,right-handed beta-helix,1.000,3863_A_6_164_549,VVSVLDFYLGTDPDYTNALNRALTASNGVLVPQGEYSTSLLSHPTC...,MSSGCGDVLSLADLQTAKKHQIFEAEVITGKSGGVAGGADIDYATN...
1,phagedepo__4467,right-handed beta-helix,1.000,4467_A_5_294_636,GVNVLDFMTFAERAAALNYTGTNDNSEAFRKALATGSRQIIAPPGR...,MDQEIKTVIQYPTGSTEFDIPFDYLSRKFVRVSLVSDDNRRLLSNI...
2,phagedepo__4576,right-handed beta-helix,1.000,4576_A_3_184_534,ELKSARFYGVSSTRTDNAVQLNAAANSGGRVVIPGGVTLFNSEILT...,MADILVTSPYRPFTLPNQFKAVFNGSIYCGTVDAVDPSNSQVQVYK...
3,phagedepo__5510,right-handed beta-helix,0.997,5510_A_5_278_575,QSISSTLEIRECTGVEVHRASGLMACFLFRGCHFCKMVDADNPSGG...,MSSGCGDVLSLNDLQIAKKHQIFEAEVITGKQGGVAGGADIDYATN...
4,phagedepo__6442,right-handed beta-helix,0.995,6442_A_6_361_681,PDNVKNQNLISHLIISNSSGIDVFYPKATFGSYESFKNNNVKFWYP...,MTVSIEVDHNDYIGNGVTTSFPYTFRIFKKSDLVVQVADLSENITE...
...,...,...,...,...,...,...
699,phagedepo__4646,right-handed beta-helix,1.000,4646_A_5_379_783,PTATDILSSNGKTLQYLTDKVNQVYDVKDFGAVGDGTTDDTAAVQS...,MSVGLYGDGVSESQENINVTQYGWTNEDVAGITLIQDYLNQIQTLF...
700,phagedepo__6447,right-handed beta-helix,1.000,6447_A_5_337_699,DRVNPFMFGALGGSNDDTIPVQSCVDSGKATQLTDAHYVSNIQLKY...,MGYFQMTRNVEELFGGVITAPHQIPFTYKSNVGGETFLSLPFYPVT...
701,phagedepo__1994,right-handed beta-helix,1.000,1994_A_5_283_679,PISWATQFGATKDGGREDVINAAIARMSQWSSFGIDPGIKFNNPLI...,MSDCKSYVSKEDLQALKESQQHIEHVARSRNAAGEKALQVTDAIRG...
702,phagedepo__4581,right-handed beta-helix,0.872,full_protein,MLQVKDFSGATHAEQIQNAINAASTSALHKTVQLEENKDYSITAPI...,MLQVKDFSGATHAEQIQNAINAASTSALHKTVQLEENKDYSITAPI...


***
## B.

In [1]:
path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/seekfold_phagedepo"
import pandas as pd
import os 

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]

header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]

depo_ppt = {}
very_good = 0
good = 0
decent = 0 
passable = 0
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    try : 
        if results_df["prob"][0] > 0.1 :
            depo_ppt[results_df["query"][0]] = (results_df["target"][0] , results_df["prob"][0])
            if results_df["prob"][0] > 0.9 :
                very_good += 1
            elif results_df["prob"][0] > 0.5 :
                good += 1
            elif results_df["prob"][0] > 0.25 :
                decent += 1
            else :
                passable += 1
                
    except IndexError :
        pass

import pprint
#pp = pprint.PrettyPrinter(width = 100, sort_dicts = True, compact = True)


print(f"So far, out of {len((outputs))} proteins searched, {len(depo_ppt)} seemed to be positive in the phagedepo dataset ({len(depo_ppt)/len(outputs)} %).\nThe proportion of very good is {very_good/len(depo_ppt)}\nThe proportion of good is {good/len(depo_ppt)}\nThe proportion of decent is {decent/len(depo_ppt)}\nThe rest is {passable/len(depo_ppt)}")


So far, out of 1926 proteins searched, 1058 seemed to be positive in the phagedepo dataset (0.54932502596054 %).
The proportion of very good is 0.7372400756143668
The proportion of good is 0.16729678638941398
The proportion of decent is 0.045368620037807186
The rest is 0.0500945179584121


***
## C.

In [2]:
path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/seekfold_phagedepo"
import pandas as pd
import os 

path_info = f"/media/concha-eloko/Linux/depolymerase_building/depolymerase_fold.csv"
info_df = pd.read_csv(path_info , sep = "\t", header = 0)
dico_folds = {"jelly-roll" : [],
              "alpha/alpha toroid" : [],
              "right-handed beta-helix" : [] ,
              "TIM beta/alpha-barrel" : [],
              "6-bladed beta-propeller" : [] ,
              "Flavodoxin-like" : [] ,
              "Alpha/Beta hydrolase fold" : [] ,
              "Other" : [],
             }

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]
header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]
depo_results = {}
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    for _,row in results_df.iterrows() :
        fold = info_df[info_df["ID"] == row["target"]]["X_Group_Name"].values[0]
        if row["prob"] >= 0.2 :
            if row["query"] not in dico_folds[fold] :
                dico_folds[fold].append(row["query"])
        elif fold == "right-handed beta-helix" and row["prob"] >= 0.2 :
            if row["query"] not in dico_folds[fold] :
                dico_folds[fold].append(row["query"])
                
for fold in dico_folds : 
    print(f"The {fold} presented {len(dico_folds[fold])} depolymerases.\n")

The jelly-roll presented 179 depolymerases.

The alpha/alpha toroid presented 1 depolymerases.

The right-handed beta-helix presented 828 depolymerases.

The TIM beta/alpha-barrel presented 9 depolymerases.

The 6-bladed beta-propeller presented 150 depolymerases.

The Flavodoxin-like presented 0 depolymerases.

The Alpha/Beta hydrolase fold presented 0 depolymerases.

The Other presented 6 depolymerases.



In [6]:
dico_folds["6-bladed beta-propeller"]

['2641.pdb',
 '1343.pdb',
 '1021.pdb',
 '5938.pdb',
 '1020.pdb',
 '1436.pdb',
 '5567.pdb',
 '16.pdb',
 '4299.pdb',
 '6707.pdb',
 '2588.pdb',
 '1848.pdb',
 '6542.pdb',
 '660.pdb',
 '3237.pdb',
 '3706.pdb',
 '5777.pdb',
 '5126.pdb',
 '1695.pdb',
 '11.pdb',
 '2591.pdb',
 '5590.pdb',
 '5724.pdb',
 '4229.pdb',
 '1721.pdb',
 '3944.pdb',
 '5790.pdb',
 '3151.pdb',
 '4892.pdb',
 '3754.pdb',
 '3123.pdb',
 '5343.pdb',
 '3557.pdb',
 '5386.pdb',
 '4178.pdb',
 '785.pdb',
 '5278.pdb',
 '4151.pdb',
 '6548.pdb',
 '2635.pdb',
 '2667.pdb',
 '6230.pdb',
 '2786.pdb',
 '5556.pdb',
 '2011.pdb',
 '3498.pdb',
 '5396.pdb',
 '6332.pdb',
 '5229.pdb',
 '657.pdb',
 '1109.pdb',
 '3857.pdb',
 '6552.pdb',
 '6693.pdb',
 '369.pdb',
 '5631.pdb',
 '3174.pdb',
 '1985.pdb',
 '6508.pdb',
 '51.pdb',
 '1043.pdb',
 '319.pdb',
 '6550.pdb',
 '1344.pdb',
 '3862.pdb',
 '1986.pdb',
 '5589.pdb',
 '814.pdb',
 '2753.pdb',
 '3166.pdb',
 '5629.pdb',
 '5791.pdb',
 '2602.pdb',
 '6046.pdb',
 '183.pdb',
 '732.pdb',
 '2764.pdb',
 '221.pdb',
 

In [6]:
info_df[info_df["X_Group_Name"] == "TIM beta/alpha-barrel" ]

,Activity,ID,X_Group_Name
7,Chitinase II,AF-A0A089GJA0-F1-model_v4_A_2_186_447.pdb,TIM beta/alpha-barrel
16,Chitinase II,AF-A0A166VHV0-F1-model_v4.pdb,TIM beta/alpha-barrel
19,Chitinase II,AF-A0A202B6A7-F1-model_v4.pdb,TIM beta/alpha-barrel
36,Hyaluronidase,AF-C0HLL5-F1-model_v4.pdb,TIM beta/alpha-barrel
40,Hyaluronidase,AF-P49370-F1-model_v4.pdb,TIM beta/alpha-barrel
41,Hyaluronidase,AF-P49371-F1-model_v4.pdb,TIM beta/alpha-barrel


In [5]:
info_df["X_Group_Name"].unique()

array(['jelly-roll', 'alpha/alpha toroid', 'right-handed beta-helix',
       'TIM beta/alpha-barrel', '6-bladed beta-propeller',
       'Alpha/Beta hydrolase fold', 'Other', 'Flavodoxin-like'],
      dtype=object)

In [12]:
path_project = "/media/concha-eloko/Linux/depolymerase_building"
dir_out = f"{path_project}/seekfold_phagedepo"
path_pdb = f"{path_project}/phagedepo_out"

import pandas as pd
import os 

outputs = [f"{dir_out}/{file}" for file in os.listdir(dir_out) if file[-3:]=="out"]

header_seekfold = ["query","target","pident","alnlen","gapopen","qstart","qend","tstart","tend","bits","prob"]


paths_decipher = [] 
good_phagedepo = {}
decipher_phagedepo = {}
for results in outputs :
    results_df = pd.read_csv(f"{results}", sep = "\t" , names = header_seekfold)
    try : 
        if results_df["prob"][0] >= 0.5 :
            good_phagedepo[results_df["query"][0]] = (results_df["target"][0] , results_df["prob"][0])
        elif results_df["prob"][0] >= 0.25 :
            decipher_phagedepo[results_df["query"][0]] = (results_df["target"][0] , results_df["prob"][0])
            paths_decipher.append(f"{path_pdb}/{results_df['query'][0]}")
    except Exception as e :
        print(e)
        pass


index 0 is out of bounds for axis 0 with size 0
index 0 is out of bounds for axis 0 with size 0


In [7]:
pp = pprint.PrettyPrinter(width = 100, sort_dicts = True, compact = True)

pp.pprint(decipher_phagedepo)

{'1043.pdb': ('AF-A0A202B6A7-F1-model_v4.pdb', 0.353),
 '1125.pdb': ('AF-A0A4V2M274-F1-model_v4_A_1_91_268.pdb', 0.353),
 '1574.pdb': ('A1l_00058_A_1_129_274.pdb', 0.473),
 '1810.pdb': ('AF-A0A0A6QEK9-F1-model_v4.pdb', 0.411),
 '1842.pdb': ('AF-A0A7K3HGV9-F1-model_v4.pdb', 0.353),
 '1873.pdb': ('AF-A0A0F6AK51-F1-model_v4.pdb', 0.277),
 '1918.pdb': ('AF-A0A0D3A0D6-F1-model_v4.pdb', 0.326),
 '2001.pdb': ('AF-A0A4V2M274-F1-model_v4_A_1_91_268.pdb', 0.442),
 '2048.pdb': ('A1l_00058_A_1_129_274.pdb', 0.411),
 '2170.pdb': ('7fhv.pdb_B', 0.277),
 '2190.pdb': ('AF-A0A4V2M274-F1-model_v4_A_1_91_268.pdb', 0.382),
 '2217.pdb': ('AF-A0A1E5LEG1-F1-model_v4.pdb', 0.473),
 '2588.pdb': ('AF-A0A7W9JCS6-F1-model_v4_A_1_95_273.pdb', 0.473),
 '2617.pdb': ('AF-A0A562WF65-F1-model_v4.pdb', 0.326),
 '2641.pdb': ('AF-A0A7G2DEW9-F1-model_v4.pdb', 0.254),
 '2691.pdb': ('AF-A0A0A6QEK9-F1-model_v4.pdb', 0.442),
 '2753.pdb': ('AF-A0A7C4BP91-F1-model_v4.pdb', 0.473),
 '2786.pdb': ('AF-A0A7G2DEW9-F1-model_v4.pdb', 0

***
# Make the CAZY DB

In [ ]:
# Create the database : 

path_pdb_ref = "/media/concha-eloko/Linux/depolymerase_building/pdb_depolymerase_domains"

# Generates the database :

foldseek createdb \
/media/concha-eloko/Linux/depolymerase_building/pdb_depolymerase_domains \
/media/concha-eloko/Linux/depolymerase_building/RefDepo_domains/RefDepo_domains_db

# Generate index files : 

foldseek createindex \
/media/concha-eloko/Linux/depolymerase_building/RefDepo_domains/RefDepo_domains_db \ 
/media/concha-eloko/Linux/depolymerase_building/tmp

# Move to server :
rsync -avzhe ssh \
/media/concha-eloko/Linux/depolymerase_building/RefDepo_domains \
conchae@garnatxa.srv.cpd:/home/conchae/depolymerase
    
rsync -avzhe ssh \
/media/concha-eloko/Linux/depolymerase_building/pdb_depolymerase_domains \
conchae@garnatxa.srv.cpd:/home/conchae/depolymerase